In [1]:
import csv
import base64
import time
import fnmatch
from datetime import datetime, timedelta
from github import Github, StatsContributor
from sustainbeat import awesome2py
from urllib.parse import urlparse 
from os import getenv, path
from dotenv import load_dotenv

In [2]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        time.sleep(1)
        t -= 1
    print('\n\n\n\n\n')

In [3]:
load_dotenv()
g = Github(getenv("GITHUB"))

In [4]:
repo_dict = awesome2py.AwesomeList(".awesome.md")
print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Appropedia - Shares knowledge to build rich, sustainable lives. [https://www.appropedia.org/Welcome_to_Appropedia] - Awesome Clean Tech - Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities. [https://github.com/nglgzz/awesome-clean-tech] - Awesome Open Climate Science - Awesome Open Atmospheric, Ocean, and Climate Science. [https://github.com/pangeo-data/awesome-open-climate-science] - Awesome Earth - "What can I do about the climate crisis?" Here are 326 things you can do. [https://github.com/philsturgeon/awesome-earth] - Awesome Open Geoscience - Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome. [https://github.com/softwareunderground/awesome-open-geoscience] - Datasets APIs and open source projects related to Climate Change - A curated list of 

In [ ]:
csv_github_projects = open('./csv/github_projects.csv', 'w', newline='') 
csv_github_fieldnames = ['project_name','oneliner','project_namespace','project_url', 'topics', 'rubric', 'last_update', 'stargazers_count','dominating_languages', 'organization', 'languages', 'homepage', 'created' , 'license','total_commits_last_year','last_issue_closed','community_development_distribution_score', 'issue_labels','good_first_issue','contributors', 'organization_avatar', 'funding' ]
writer_github_projects = csv.DictWriter(csv_github_projects, fieldnames=csv_github_fieldnames)
writer_github_projects.writeheader()
                                    
csv_github_organizations = open('./csv/github_organizations.csv', 'w', newline='') 
csv_github_organizations_fieldnames =  ['organization_name','organization_avatar','organization_public_repos','organization_location','organization_description','organization_public_members','organization_repos_count','total_organization_stars' ]
writer_github_organizations = csv.DictWriter(csv_github_organizations, fieldnames=csv_github_organizations_fieldnames)
writer_github_organizations.writeheader()
organization_list = []

csv_gitlab_projects = open('./csv/gitlab_projects.csv', 'w', newline='') 
csv_gitlab_fieldnames = ['project_name','oneliner','project_namespace','project_url', 'rubric']
writer_gitlab_projects = csv.DictWriter(csv_gitlab_projects, fieldnames=csv_gitlab_fieldnames)
writer_gitlab_projects.writeheader()

csv_custom_platform_projects = open('./csv/custom_platform_projects.csv', 'w', newline='') 
csv_custom_platform_fieldnames = ['project_name','oneliner','project_namespace','project_url', 'rubric']
writer_custom_platform_projects = csv.DictWriter(csv_custom_platform_projects, fieldnames=csv_custom_platform_fieldnames)
writer_custom_platform_projects.writeheader()


for r in repo_dict.rubrics:        
    for e in r.entries: 
        print("------------------------")
        print("Processing %s" % e.name)
        print("URL: %s" % e.url)
        if urlparse(e.url).netloc == 'github.com':
            print("%s is a GitHub project" % e.name)
            try:

                remaining,limit  = g.rate_limiting
                resettime = g.rate_limiting_resettime

                if(remaining<20):
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp() 
                    countdown((int(resettime)-int(current_time)+2))

                print("GitHub Requests | Limit: {}, Remaining: {}".format(limit, remaining))
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(e.url).path.strip("/")
                user, project_name  = path.split(repo_path)
                repo = g.get_repo(repo_path)
                files = repo.get_contents("")

                # Check if the project is still active 
                issues = repo.get_issues(state='closed')

                ## preprocessing
                if issues.totalCount > 0:
                    last_issue_closed = issues[0].updated_at
                    
                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_delta = datetime.utcnow() - last_issue_closed
                
                             # Do not process inactive projects
                if last_issue_delta > timedelta(days=365) and total_commits_last_year == 0:
                    print("%s is an inactive project" % e.name)
                    print("Skip processing")
                    print("------------------------")
                    continue
                
                
                try: 
                    license = repo.get_license()
                    if license.license.spdx_id == 'NOASSERTION':
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"



                    

                labels = (",".join([entry.name for entry in repo.get_labels()])) 
                topics = (",".join(repo.get_topics())) 

                languages_states = repo.get_languages()
                programming_languages = (",".join(languages_states.keys()))
                dominating_languages = list(languages_states.keys())[0]

                try:
                    funding_file = repo.get_contents(".github")
                    for file in funding_file:
                        if file.path==".github/FUNDING.yml":
                            print("Funding found")
                            funding_information = base64.b64decode(file.content)
                        else:
                            funding_information = None
                except:
                    print("No funding information found")
                    funding_information = None


                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(sorted(contributor_activity.items(), key=lambda item: item[1]))
                weighted_contribution = {k: v / commits_total for k, v in sorted_contributor.items()}
                
                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(weighted_contribution.values())

   

                 # Gathering organization data 
                if repo.organization is None: 
                    organization_name = None

                elif repo.organization.login not in organization_list: 
                    organization_name = repo.organization.login
                    organization_avatar = repo.organization.avatar_url
                    organization_public_repos = repo.organization.public_repos
                    organization_location = repo.organization.location
                    organization_description = repo.organization.description
                    organization_repos = repo.organization.get_repos()
                    organization_public_members_count = 0
                    
                    
                    total_organization_stars = 0
                    for k in organization_repos:
                        total_organization_stars = total_organization_stars + k.stargazers_count

                    organization_data = {  'organization_name': organization_name,
                                           'organization_avatar': organization_avatar,
                                           'organization_public_repos': organization_public_repos,
                                           'organization_location': organization_location,
                                           'organization_description': organization_description,
                                           'organization_public_members': organization_public_members_count,
                                           'organization_repos_count': organization_repos.totalCount,
                                           'total_organization_stars': total_organization_stars,           
                                         }

                    organization_list.append(organization_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)
                

                else:
                    organization_name = ''
                    organization_avatar = ''
                
                
                project_data = {  'project_name': e.name,
                                  'project_namespace': user,
                                  'project_url': repo.clone_url,
                                  'rubric':r.key,
                                  'oneliner':e.text[2:],
                                  'topics':topics,
                                  'organization':organization_name,
                                  'created':repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'last_update':repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'total_commits_last_year':total_commits_last_year,
                                  'last_issue_closed':last_issue_closed.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'community_development_distribution_score':community_development_distribution_score,
                                  'stargazers_count':repo.stargazers_count,
                                  'dominating_languages':dominating_languages,
                                  'languages':programming_languages,
                                  'homepage':repo.homepage,
                                  'issue_labels': labels,
                                  'good_first_issue':repo.get_issues(state='open',labels=['good first issue']).totalCount,
                                  'license':license_name,
                                  'contributors':repo.get_contributors().totalCount,
                                  'funding': funding_information,
                                }

                print("Project Data:")
                print(project_data)
                writer_github_projects.writerow(project_data)

            except Exception as e:
                print(e)
                
        elif urlparse(e.url).netloc == 'gitlab.com':
            print("%s is a Gitlab project" % e.name)   
            repo_path = urlparse(e.url).path.strip("/")
            user, project_name  = path.split(repo_path)
            
            project_data = {  'project_name': e.name,
                              'project_namespace': user,
                              'project_url': repo_path,
                              'rubric':r.key,
                              'oneliner':e.text[2:],
                           }
            print("Project Data:")
            print(project_data)
            writer_gitlab_projects.writerow(project_data)
            
        else:
            print("%s is hosted on custom platform" % e.name)   
            repo_path = urlparse(e.url).path.strip("/")
            user, project_name  = path.split(repo_path)
            
            project_data = {  'project_name': e.name,
                              'project_namespace': user,
                              'project_url': repo_path,
                              'rubric':r.key,
                              'oneliner':e.text[2:],
                           }
            
            print("Project Data:")
            print(project_data)
            writer_custom_platform_projects.writerow(project_data)

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 4996
Organization Data:
{'organization_name': 'pvlib', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_public_repos': 4, 'organization_location': None, 'organization_description': None, 'organization_public_members': 0, 'organization_repos_count': 4, 'total_organization_stars': 634}
Project Data:
{'project_name': 'pvlib-python', 'project_namespace': 'pvlib', 'project_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewables,photovoltaic', 'organization': 'pvlib', 'created': '2015/02/17, 00:21:33', 'last_update': '2021/10/28, 16:50:39', 'total_commits_last_year': 100, 'last_issu

{'project_name': 'feedinlib', 'project_namespace': 'oemof', 'project_url': 'https://github.com/oemof/feedinlib.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Contains implementations of photovoltaic models to calculate electricity generation from a PV installation based on given solar radiation. Furthermore it contains all necessary pre-calculations.', 'topics': '', 'organization': 'oemof', 'created': '2014/07/24, 09:03:29', 'last_update': '2021/08/21, 07:58:55', 'total_commits_last_year': 43, 'last_issue_closed': '2021/10/12, 15:25:58', 'community_development_distribution_score': 0.6876456876456876, 'stargazers_count': 64, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': '', 'issue_labels': 'bug,documentation,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 10, 'funding': None}
------------------------
Processing photovoltaic
URL: https://github.com/pvedu/photovoltaic
photovoltaic 

{'project_name': 'bifacial_radiance', 'project_namespace': 'NREL', 'project_url': 'https://github.com/NREL/bifacial_radiance.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Toolkit for working with RADIANCE for the ray-trace modeling of Bifacial Photovoltaics.', 'topics': 'radiance,bifacial,photovoltaics,renewable-energy,renewables,gui', 'organization': '', 'created': '2017/12/13, 17:56:18', 'last_update': '2021/10/25, 22:59:29', 'total_commits_last_year': 4, 'last_issue_closed': '2021/10/28, 17:17:27', 'community_development_distribution_score': 0.29161118508655126, 'stargazers_count': 50, 'dominating_languages': 'Python', 'languages': 'Python,C++,TeX,C,Makefile,Awk', 'homepage': 'https://bifacial-radiance.readthedocs.io', 'issue_labels': 'bug,contribution,dependencies,documentation,duplicate,enhancement,Fixed in development branch,gencumsky,geometry,good first issue,help wanted,HPC,invalid,JOSS,linux,Pvlib,question,v0_3_3,v0_3_4,v0_3_5,Wishlists,wontfix', 'good_first_i

{'project_name': 'SolarTherm', 'project_namespace': 'SolarTherm', 'project_url': 'https://github.com/SolarTherm/SolarTherm.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Solar thermal power/fuel station performance simulation and optimization using Modelica.', 'topics': 'engineering,energy,solar,thermodynamics,optimisation,simulation,modelica,modelica-library', 'organization': 'SolarTherm', 'created': '2015/07/21, 05:23:10', 'last_update': '2021/10/22, 12:59:09', 'total_commits_last_year': 430, 'last_issue_closed': '2021/10/22, 12:06:28', 'community_development_distribution_score': 0.6436031331592689, 'stargazers_count': 20, 'dominating_languages': 'Modelica', 'languages': 'Modelica,Python,C,Shell', 'homepage': '', 'issue_labels': 'bug,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'LGPL-3.0', 'contributors': 8, 'funding': None}
------------------------
Processing LibreSolar
URL: https://github.com/LibreSolar/bms-firmware


{'project_name': 'pvanalytics', 'project_namespace': 'pvlib', 'project_url': 'https://github.com/pvlib/pvanalytics.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Quality control, filtering, feature labeling, and other tools for working with data from photovoltaic energy systems.', 'topics': '', 'organization': '', 'created': '2020/02/18, 17:10:34', 'last_update': '2021/07/05, 07:27:46', 'total_commits_last_year': 16, 'last_issue_closed': '2021/05/13, 13:11:16', 'community_development_distribution_score': 0.10370370370370374, 'stargazers_count': 35, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': 'https://pvanalytics.readthedocs.io', 'issue_labels': 'bug,ci,dependency,documentation,duplicate,enhancement,feature,good first issue,help wanted,invalid,question,tests,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 5, 'funding': None}
------------------------
Processing IonMonger
URL: https://github.com/PerovskiteSCModelling/IonMonger
Io

...

{'project_name': 'WecOptTool', 'project_namespace': 'SNL-WaterPower', 'project_url': 'https://github.com/SNL-WaterPower/WecOptTool.git', 'rubric': 'Hydro Energy', 'oneliner': 'Allows users to perform wave energy converter device design optimization studies with constrained optimal control.', 'topics': 'snl-applications', 'organization': 'SNL-WaterPower', 'created': '2021/06/08, 17:44:46', 'last_update': '2021/09/08, 15:36:42', 'total_commits_last_year': 11, 'last_issue_closed': '2021/07/08, 21:30:53', 'community_development_distribution_score': 0.36363636363636365, 'stargazers_count': 2, 'dominating_languages': 'TeX', 'languages': 'TeX,Python,GLSL', 'homepage': '', 'issue_labels': 'bug,documentation,duplicate,enhancement,good first issue,help wanted,invalid,question,testing,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 3, 'funding': None}
------------------------
Processing CACTUS
URL: https://github.com/SNL-WaterPower/CACTUS
CACTUS is a GitHub project
GitHub Reque

{'project_name': 'WEC-Sim', 'project_namespace': 'WEC-Sim', 'project_url': 'https://github.com/WEC-Sim/WEC-Sim.git', 'rubric': 'Hydro Energy', 'oneliner': 'Wave Energy Converter Simulator is an open source code for simulating wave energy converters.', 'topics': 'wec-sim,marine-renewable-energy,wave-energy,matlab,simulink,snl-applications,hydrodynamics', 'organization': 'WEC-Sim', 'created': '2014/06/03, 16:56:49', 'last_update': '2021/10/28, 17:00:19', 'total_commits_last_year': 254, 'last_issue_closed': '2021/10/28, 16:38:43', 'community_development_distribution_score': 0.677570093457944, 'stargazers_count': 75, 'dominating_languages': 'MATLAB', 'languages': 'MATLAB', 'homepage': 'http://wec-sim.github.io/WEC-Sim', 'issue_labels': 'Added Mass,BEM/BEMIO,Body Class,Body-to-body,Bug,Cable,Constraint,Control,Documentation,Drag/Morison,Enhancement,Example,Git/GitHub,Good first issue,High,Library,MATLAB/Simulink,MCR/PCT,Mesh,MoorDyn,Mooring,Multi-body solver,Non-linear hydro,PTO,PTO-Sim,Res

{'project_name': 'FEHM', 'project_namespace': 'lanl', 'project_url': 'https://github.com/lanl/FEHM.git', 'rubric': 'Geothermal Energy', 'oneliner': 'Has proved to be a valuable asset on a variety of projects of national interest including Environmental Remediation of the Nevada Test Site, the LANL Groundwater Protection Program, geologic CO2 sequestration, Enhanced Geothermal Energy programs, Oil and Gas production, Nuclear Waste Isolation, and Arctic Permafrost.', 'topics': 'porous-flow,simulation-modeling,reactive-chemistry,coupled-heat-mass,multiphase-transport,groundwater-modelling,carbon-storage,soil-vapor,earth-science,carbon-sequestration,subsurface-remediation,nuclear-waste-repositories,subsurface-hydrology,geothermal-energy', 'organization': 'lanl', 'created': '2017/12/13, 21:46:09', 'last_update': '2021/10/06, 07:15:23', 'total_commits_last_year': 4, 'last_issue_closed': '2020/10/22, 22:15:38', 'community_development_distribution_score': 0.1544943820224719, 'stargazers_count'

{'organization_name': 'alexliberzonlab', 'organization_avatar': 'https://avatars.githubusercontent.com/u/2659993?v=4', 'organization_public_repos': 87, 'organization_location': 'Tel Aviv University, Tel Aviv, Israel', 'organization_description': None, 'organization_public_members': 0, 'organization_repos_count': 87, 'total_organization_stars': 10}
Project Data:
{'project_name': 'Multiscale_Ulva', 'project_namespace': 'alexliberzonlab', 'project_url': 'https://github.com/alexliberzonlab/Multiscale_Ulva.git', 'rubric': 'Bioenergy', 'oneliner': 'A multi-reactor, algae farm, simulation base function that will be solved in time.', 'topics': '', 'organization': 'alexliberzonlab', 'created': '2020/10/01, 16:56:19', 'last_update': '2021/10/14, 11:42:42', 'total_commits_last_year': 8, 'last_issue_closed': '2020/10/01, 17:41:08', 'community_development_distribution_score': 0.11538461538461542, 'stargazers_count': 2, 'dominating_languages': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Pytho

{'organization_name': 'TRI-AMDD', 'organization_avatar': 'https://avatars.githubusercontent.com/u/62159728?v=4', 'organization_public_repos': 12, 'organization_location': None, 'organization_description': None, 'organization_public_members': 0, 'organization_repos_count': 12, 'total_organization_stars': 113}
Project Data:
{'project_name': 'beep', 'project_namespace': 'TRI-AMDD', 'project_url': 'https://github.com/TRI-AMDD/beep.git', 'rubric': 'Battery', 'oneliner': 'A set of tools designed to support Battery Evaluation and Early Prediction of cycle life corresponding to the research of the d3batt program and the Toyota Research Institute.', 'topics': '', 'organization': 'TRI-AMDD', 'created': '2020/02/13, 23:09:45', 'last_update': '2021/10/28, 16:41:11', 'total_commits_last_year': 816, 'last_issue_closed': '2021/10/28, 14:02:43', 'community_development_distribution_score': 0.6222583265637693, 'stargazers_count': 48, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': '